<img style="float: left;" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Functions Semantic  

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and what their descriptions  
* Listing types of decision-making frameworks dynamically based on the type  

Semantic Kernel Functions are the core building blocks of functionality for intelligent AI orchestration. Semantic Kernel functions usually have a single responsibility to perform. For example, a Sementic Kernel function can: send an e-mail, call an API, recommend a reasoning framework for a high-stakes decision etc. 

What makes Semantic Kernel functions so special? Can't one just produce similar outcomes by using a prompt in an LLM or just writing a software program? In order to answer that, one has to look at what makes GenAI so innovative. The GenAI models have a unique ability to process instructions with reasoning and logic. This allows these models to behave almost like a human decision maker. Even with basic prompts, GenAI models perform reasonably well. However, providing GenAI models with additional functions allows them to gain access to business processes, data and basically anything a GenAI model should consider when performing AI orchestration. 

For example, imagine you want to prepare a great Thanksgiving dinner and want to get help from a GenAI cooking application to create a new recipe. You enter what you want todo and it comes up with the most delicious looking feast for Thanksgiving. However, there is a problem it used ingredients and recommended using cooking appliances that you do not own! You could enter all of the ingredients and keep prompting until it narrowed down the what you could realistically make. Wouldn't it be better if the GenAI cooking application had access to: your available ingredients, your time availability, kitchen appliances and even your allergic preferences. Now the GenAI model can craft a Thanksgiving feast understanding the parameters and data without having to be guided. This is exactly what Semantic Kernel functions provide.

Semantic Kernel functions come in two flavors: semantic functions and native functions. Semantic functions are basically prompt instructions that help guide GenAI LLMs around the specific guidelines it should adhere to when building the AI orchestration. Therefore, semantic functions are likely the first place most AI architects/engineers start when composing AI orchestration with Semantic Kernel. Semantic functions can also take in parameters for dynamic creaation of instructions. 

Below is an example of semantic function that can be used for facilitating decision-making. Note the {{$decisionToMake}} input where the parameter can be dynamically passed in to provide additional specificity.

```javascript
[DECISION FRAMEWORKS TO USE]
Price's Law
Pareto Principle
Laplace Rule of Succession
Eisenhower Matrix
Median Rule of Five
Second Order Thinking

[DECISION GUIDANCE]
Try to use the best fitting framework.  
Prefer using quantitative decision frameworks rather qualitative ones.  
Use the code interpreter to validate quantitative reasoning.

Use the decision frameworks above to help the user make the following decision:
{{$decisionToMake}}
```  

The image below illustrates the core value of semantic functions. 

<img width ="700px" src="https://learn.microsoft.com/en-us/semantic-kernel/media/semantic-function-explainer.png">

 
 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.13.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.13.0-alpha"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.SemanticKernel, 1.13.0 Microsoft.SemanticKernel.Plugins.Core, 1.13.0-alpha

In [2]:
Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Using Azure OpenAI Service


### Step 2 - Create a Semantic Kernel - Semantic Function

Semantic functions are constructs inside Semantic Kernel that are basically just prompt instructions. What makes the special is that they can be exposed as functions, thus it makes re-usability easy. Furthermore, future modules will show how they can faciliate composability into plugins. 

In [3]:
var decisionPromptTemplate = """
Provide a list of decision frameworks that can help improve the quality of decisions.
""";

// Takes a regular prompt and creats a function that can be used to invoke the model
var semanticDecisionFunction = semanticKernel.CreateFunctionFromPrompt(decisionPromptTemplate);

await foreach (var streamChunk in semanticKernel.InvokeStreamingAsync(semanticDecisionFunction))
{
   Console.Write(streamChunk);
}

Certainly! Here are several decision frameworks that can help improve the quality of decisions:

1. **SWOT Analysis**
   - Strengths
   - Weaknesses
   - Opportunities
   - Threats
   SWOT helps in understanding both internal and external factors that can impact the decision.

2. **Pros and Cons List**
   - A simple yet effective tool for weighing the advantages and disadvantages of different options.

3. **Cost-Benefit Analysis (CBA)**
   - Evaluate the total expected costs versus the total expected benefits of one or more actions to choose the most beneficial option.

4. **Decision Matrix (or Pugh Matrix)**
   - A quantitative approach to evaluate and prioritize a list of options against a set of criteria.

5. **Rational Decision-Making Model**
   - Define the problem
   - Identify the decision criteria
   - Allocate weights to the criteria
   - Develop alternatives
   - Evaluate the alternatives
   - Select the best alternative
   This model is systematic and analytical, making it u

### Step 3 - Semantic Function Dynamic Execution

Semantic functions can be dynamically composed using parameter placeholders, which can then be dynamically rendered using Kernel arguments. This allows the ease of passing in parameters and execution settings into the semantic functions.

Execute the cell below to view how the prompt can dynamically instruct the LLM to retrieve different types of decision-making frameworks.  

> "If the only tool you have is a hammer, you tend to see every problem as a nail."
>
> -- <cite>Abraham Maslow (renowned American psychologist)</cite> 

In [5]:
// Create the OpenAI prompt execution settings
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1500, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

// Takes a prompt with input variables and creates a function that can be used to invoke the model
var decisionPromptTemplate = """
Provide a list of {{$numberOfFrameworks}} frameworks that can help improve the quality of {{$decisionType}} decisions.
""";
var semanticDecisionFunctionWithInput = semanticKernel.CreateFunctionFromPrompt(decisionPromptTemplate, openAIPromptExecutionSettings);

// Dynamically set the number of frameworks and decision type -> strategic
var kernelArguments = new KernelArguments();
kernelArguments.Add("numberOfFrameworks", 3);
kernelArguments.Add("decisionType", "strategic");

// Stream the output of the model
await foreach (var streamChunk in semanticKernel.InvokeStreamingAsync(semanticDecisionFunctionWithInput, kernelArguments))
{
   Console.Write(streamChunk);
}

Certainly! Here are three frameworks that can help improve the quality of strategic decisions:

1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**:
   - **Strengths**: Identify internal attributes and resources that support a successful outcome.
   - **Weaknesses**: Recognize internal attributes and resources that could work against a successful outcome.
   - **Opportunities**: Identify external factors that the organization can capitalize on or use to its advantage.
   - **Threats**: Recognize external factors that could cause trouble for the organization or project.

2. **Porter's Five Forces**:
   - **Competitive Rivalry**: Assess the intensity of competition among existing competitors in the market.
   - **Threat of New Entrants**: Evaluate how easy or difficult it is for new competitors to enter the market.
   - **Bargaining Power of Suppliers**: Determine the power suppliers have to drive up prices.
   - **Bargaining Power of Buyers**: Assess the power customers

> "Decision frameworks are like maps. Use them to navigate complex decision-making terrain.  
>
> -- <cite>Unknown</cite> 

In [7]:
// Dynamically set the number of frameworks and decision type -> requring fast action
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings);
kernelArguments.Add("numberOfFrameworks", 5);
kernelArguments.Add("decisionType", "requring fast action");

// Stream the output of the model
await foreach (var streamChunk in semanticKernel.InvokeStreamingAsync(semanticDecisionFunctionWithInput, kernelArguments))
{
   Console.Write(streamChunk);
}

Certainly! Here are five frameworks that can help improve the quality of decisions requiring fast action:

1. **OODA Loop (Observe, Orient, Decide, Act)**:
   - **Observe**: Gather information and data from the environment.
   - **Orient**: Analyze the information and understand the context.
   - **Decide**: Choose a course of action based on the analysis.
   - **Act**: Implement the decision quickly and efficiently.
   - This framework is particularly useful in dynamic and rapidly changing situations.

2. **PDCA Cycle (Plan-Do-Check-Act)**:
   - **Plan**: Identify the problem and develop a plan to address it.
   - **Do**: Implement the plan on a small scale to test its effectiveness.
   - **Check**: Evaluate the results of the implementation.
   - **Act**: If the plan is successful, implement it on a larger scale; if not, refine and adjust the plan.
   - This iterative process helps in making informed decisions and continuous improvement.

3. **SWOT Analysis (Strengths, Weaknesses, Op